# Mapeo catálogo RCEA a Fronteras

1. Cargar datos resultado del topic modelling
2. Filtrar:
    * SNI y vigencia
    * Responsables técnicos convocatoria
3. Match
    * area, campo, disciplina, subdisciplina

In [4]:
import pandas as pd
from joblib import load, dump
from utils import *

ModuleNotFoundError: No module named 'utils'

In [4]:
file_in   = 'data-raw/propuesta_evaluadores.pkl'
file_sni  = 'data-raw/SNI_vigentes_final.csv'
file_fron = 'data-raw/solicitudes_201912031041.csv'
file_rcea = 'data-raw/data_eval.pkl'
dics_mapeo= 'data-clean/dics_mapeo_rcea_eval.pkl'
file_out  = 'data-clean/evaluadores_RCEA_para_ciencia_frontera.csv'
top_n = 10

## Cargar datos

In [5]:
df = load(file_in)
n_test = set(df['id_proyecto'].to_list())
df.shape, df.columns

FileNotFoundError: [Errno 2] No such file or directory: 'data-raw/propuesta_evaluadores.pkl'

## Filtrar
### SNI y termino

In [4]:
def filtrar_sni(df, file_sni, file_fron):
    """
    Toma el df de topic modelling y regresasolo los sni validos adjuntando el nivel del sni
    """
    df_sni = pd.read_csv(file_sni)
    df_sni = df_sni.drop(['IVIGENCIA','FVIGENCIA'], axis=1)
    df = df.set_index('CVU').join(df_sni.set_index('CVU'), how='left').reset_index()
    df.columns = ['CVU','id_proyecto','score_tm','nivel_sni']
    
    # Filtrar snis para sinergias
    sine = pd.read_csv(file_fron)
    sine = sine[sine['modalidad']=='Modalidad Sinergia']
    sine = sine['proyecto_id'].to_list()
    sine = df[(df['id_proyecto'].isin(sine)) & (df['nivel_sni']==1)].index.to_list()
    df = df.drop(sine)
    return df

df = filtrar_sni(df, file_sni, file_fron)
print(df.shape, df.columns)
filt = set(df['id_proyecto'].to_list())
len(n_test), len(filt), len(n_test.intersection(filt))

(63872, 4) Index(['CVU', 'id_proyecto', 'score_tm', 'nivel_sni'], dtype='object')


(3232, 3232, 3232)

In [5]:
def filtrar_responsable_tecnico(df, file_fron):
    df_fron = pd.read_csv(file_fron)
    df_fron = df_fron['rt_cvu'].to_list()
    df = df[~df['CVU'].isin(df_fron)]
    return df
df = filtrar_responsable_tecnico(df, file_fron)
print(df.shape, df.columns)
filt = set(df['id_proyecto'].to_list())
len(n_test), len(filt), len(n_test.intersection(filt))

(55222, 4) Index(['CVU', 'id_proyecto', 'score_tm', 'nivel_sni'], dtype='object')


(3232, 3232, 3232)

## Match

In [6]:
def match_clasificacion(df, file_rcea, file_fron, dics_mapeo):
    clasi = ['area', 'campo', 'disciplina', 'subdisciplina']
    dic_rcea = {'01':'CIENCIAS FÍSICO MATEMÁTICAS Y CIENCIAS DE LA TIERRA', '02':'BIOLOGÍA Y QUÍMICA', 
                '03':'MEDICINA Y CIENCIAS DE LA SALUD', '04':'HUMANIDADES Y CIENCIAS DE LA CONDUCTA', 
                '05':'CIENCIAS SOCIALES', '06':'CIENCIAS AGROPECUARIAS Y BIOTECNOLOGÍA', '07':'INGENIERÍA Y TECNOLOGÍA'}
    dics_mapeo = load(dics_mapeo)
    # Cargar evaluadores rcea y mapear areas
    df_rcea = load(file_rcea)
    df_rcea.columns = ['cvu', 'nivel_sni', 'area', 'estado', 'dependencia', 'campo',
                       'disciplina', 'subdisciplina', 'nombres', 'apellidos', 'keywords']
    df_rcea = df_rcea[['cvu']+clasi].dropna(subset=['cvu', 'area'])
    df_rcea['area'] = df_rcea['area'].apply(lambda x: x.split('-')[1])
    df_rcea['area'] = df_rcea['area'].replace({'A3':'03','A6':'06','A7':'07'}).map(dic_rcea)
    for col in clasi: df_rcea[col] = df_rcea[col].map(dics_mapeo[col])
    # Cargar datos de proyectos
    df_fron = pd.read_csv(file_fron)
    df_fron = df_fron[['proyecto_id'] + clasi]
    # Unir y matchear df
    df = df.set_index('id_proyecto').join( df_fron.set_index('proyecto_id') ).reset_index()
    df = df.set_index('CVU').join( df_rcea.set_index('cvu'),  lsuffix='_proy', rsuffix='_rcea' ).reset_index()
    for col in clasi:
        df[col] = df[col+'_proy'] == df[col+'_rcea']
        df.drop([col+'_proy',col+'_rcea'], axis=1, inplace=True)
    df.columns = ['cvu','id_proyecto','score_tm','nivel_sni','m_area','m_campo','m_disciplina','m_subdisciplina']
    df = df.sort_values(['id_proyecto','score_tm'], ascending=[True,False])
    return df

df = match_clasificacion(df, file_rcea, file_fron, dics_mapeo)
print(df.shape, df.columns)
filt = set(df['id_proyecto'].to_list())
len(n_test), len(filt), len(n_test.intersection(filt))

(55222, 8) Index(['cvu', 'id_proyecto', 'score_tm', 'nivel_sni', 'm_area', 'm_campo',
       'm_disciplina', 'm_subdisciplina'],
      dtype='object')


(3232, 3232, 3232)

## Guardar

Seleccionar top_n y evaluar cobertura

In [7]:
df = df.sort_values('score_tm', ascending=False)
df = df.groupby('id_proyecto').head(top_n)
df = df.sort_values(['id_proyecto','score_tm'], ascending=[True,False])
df.groupby('id_proyecto').count()['cvu'].value_counts()

10    3225
8        3
9        2
6        2
Name: cvu, dtype: int64

Evaluar cobertura

Verificar que esten todos los proyectos

In [11]:
print(len(n_test), len(n_test.intersection(filt)), len(filt))

3232 3232 3232


Verificar match

Datos de aquellos que reportaron RCEA
* area 0.764
* campo 0.586
* disciplina 0.408
* subdisciplina 0.377

In [9]:
for col in ['area', 'campo', 'disciplina', 'subdisciplina']:
    s = df["m_"+col].value_counts()
    print(col, round(s[1]/(s[0]+s[1]),3) )

area 0.452
campo 0.133
disciplina 0.032
subdisciplina 0.01


Guardar

In [10]:
df.to_csv(file_out, index=False)

area 
 0.4630775577557756
campo 
 0.1368605610561056
disciplina 
 0.0345503300330033
subdisciplina 
 0.009797854785478548
